In [2]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np

In [3]:
path_to_data = 'C:/Users/PetukhovMD/Desktop/szp/'
path_to_docs = 'C:/Users/PetukhovMD/Desktop/справочники/'
months = ['feb.xlsx', 'jan.xlsx']
sheet = 'Sheet'

In [4]:
def to_double(row, f_name):
    if type(row[f_name]) is float:
        return row[f_name] 
    t = str(row[f_name]).split(',')
    if t[0] == 'nan':
        return 0.0
    return float(t[0]+'.'+t[1])

In [5]:
df = pd.read_excel(path_to_data + 'jan.xlsx')
df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
sample = df[(df.stv >= 1.0) & (df.status == 'Работа') & (df.day == 1) & ((df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство')) & (df.inn != 9718071371)]
sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index()
sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
sums = sums[['inn_x', 'snils', 'sum_x', 'job']].drop_duplicates(['inn_x', 'snils', 'sum_x'])
sums = sums.rename(columns = {'inn_x': 'inn', 'sum_x': 'sum_jan', 'job':'job_jan'})
sums


,inn,snils,sum_jan,job_jan
0,5003021368,001-191-281 85,106626.88,Учитель
1,5003021368,001-244-107 65,109394.87,Учитель
2,5003021368,001-263-234 77,74520.00,Воспитатель
3,5003021368,001-322-594 80,83013.52,Специалист
4,5003021368,001-519-663 10,142589.38,Учитель
...,...,...,...,...
111647,9715217689,174-783-284 11,87674.67,Учитель
111648,9715217689,185-436-935 07,103382.85,Учитель
111649,9715217689,185-540-582 90,65495.67,Воспитатель
111650,9715217689,191-336-708 73,147433.25,Учитель


In [80]:
df = pd.read_excel(path_to_data + 'feb.xlsx')
df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
sample = df[(df.stv >= 1.0) & (df.status == 'Работа') & (df.day == 1) & ((df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство')) & (df.inn != 9718071371)]
sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index()
sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
sums = sums[['inn_x', 'snils', 'sum_x', 'job']].drop_duplicates(['inn_x', 'snils', 'sum_x'])
sums = sums.rename(columns = {'inn_x': 'inn', 'sum_x': 'sum_feb', 'job':'job_feb'})

In [13]:
sums = sums.rename(columns = {'inn_x': 'inn', 'sum_x': 'sum_feb', 'job':'job_feb'})

In [6]:
res = sums

In [ ]:
res

In [14]:
res = res.merge(sums, how= 'outer', on=['inn', 'snils'])

In [15]:
res[res.sum_feb.isna()]

,inn,snils,sum_jan,job_jan,sum_feb,job_feb
11,5003021368,002-406-281 85,60058.00,Уборщик служебных и производственных помещений,NaN,NaN
20,5003021368,008-050-481 09,111511.49,Учитель,NaN,NaN
21,5003021368,008-611-548 31,62463.00,Воспитатель,NaN,NaN
27,5003021368,010-193-270 91,69520.00,Воспитатель,NaN,NaN
30,5003021368,011-088-151 00,141199.36,Учитель,NaN,NaN
...,...,...,...,...,...,...
111640,9715217689,161-422-955 47,63990.00,Учитель-логопед,NaN,NaN
111641,9715217689,166-758-433 17,131050.00,Старший воспитатель,NaN,NaN
111642,9715217689,168-197-789 36,110996.96,Воспитатель,NaN,NaN
111647,9715217689,174-783-284 11,87674.67,Учитель,NaN,NaN


In [48]:
def szp(row, months):
    if pd.isna(row['job_' + months[-1]]):
        return np.nan
    cnt = 0
    sum_szp = 0
    for month in months:
        if pd.isna(row['sum_' + month]):
            continue
        cnt += 1
        sum_szp += row['sum_' + month]
    # if row['inn'] == 9715217689 and row['snils'] == '173-260-871 66':
    #     print(cnt, sum_szp)
    return sum_szp / cnt

In [49]:
res['szp'] = res.apply(lambda row: szp(row, ['jan', 'feb']), axis= 1)

In [50]:
res

,inn,snils,sum_jan,job_jan,sum_feb,job_feb,szp,szp_ped
0,5003021368,001-191-281 85,106626.88,Учитель,139753.92,Учитель,123190.40,123190.40
1,5003021368,001-244-107 65,109394.87,Учитель,109541.13,Учитель,109468.00,109468.00
2,5003021368,001-263-234 77,74520.00,Воспитатель,81178.00,Воспитатель,77849.00,77849.00
3,5003021368,001-322-594 80,83013.52,Специалист,75815.00,Специалист,79414.26,NaN
4,5003021368,001-519-663 10,142589.38,Учитель,163810.32,Учитель,153199.85,153199.85
...,...,...,...,...,...,...,...,...
130102,9715217689,173-260-871 66,NaN,NaN,67358.00,Воспитатель,67358.00,67358.00
130103,9715217689,184-638-768 23,NaN,NaN,142971.65,Педагог-организатор,142971.65,142971.65
130104,9715217689,184-820-648 92,NaN,NaN,79250.00,Специалист по закупкам,79250.00,NaN
130105,9715217689,187-933-260 19,NaN,NaN,125727.05,Учитель,125727.05,125727.05


In [55]:
depr_month_inc = 84100.0

In [20]:
df = pd.read_excel(path_to_docs + 'input.xlsx', sheet_name='УГД')

In [24]:
df

,Должность,УГД,Педагогический,Работники непосредственно осуществляющие и обеспечивающие основной учебно-вспомогательный процесс во взаимодействии с детьми
0,Главный бухгалтер,Административно-управленческий персонал,0,0
1,Директор,Административно-управленческий персонал,0,0
2,Директор филиала,Административно-управленческий персонал,0,0
3,Заведующий,Административно-управленческий персонал,0,0
4,Заместитель главного бухгалтера,Административно-управленческий персонал,0,0
...,...,...,...,...
291,Эксперт,Прочий персонал (включая иной административный...,0,0
292,Электромеханик,Прочий персонал (включая иной административный...,0,0
293,Старший юрисконсульт,Прочий персонал (включая иной административный...,0,0
294,Ведущий юрисконсульт,Прочий персонал (включая иной административный...,0,0


In [23]:
ped = df[df['Педагогический'] == 1]['Должность'].tolist()
ped

['Ассистент',
 'Декан факультета',
 'Директор института',
 'Доцент',
 'Заведующий кафедрой',
 'Профессор',
 'Преподаватель',
 'Старший преподаватель',
 'Воспитатель',
 'Инструктор-методист',
 'Инструктор по труду',
 'Инструктор по физической культуре',
 'Концертмейстер',
 'Мастер производственного обучения',
 'Методист',
 'Музыкальный руководитель',
 'Педагог дополнительного образования',
 'Педагог-библиотекарь',
 'Педагог-организатор',
 'Педагог-психолог ',
 'Преподаватель ',
 'Преподаватель-организатор основ безопасности жизнедеятельности',
 'Руководитель физического воспитания',
 'Социальный педагог',
 'Старший вожатый',
 'Старший воспитатель',
 'Старший инструктор-методист',
 'Старший методист',
 'Старший педагог дополнительного образования',
 'Старший педагог-психолог',
 'Старший тренер-преподаватель',
 'Тренер-преподаватель',
 'Тьютор',
 'Учитель',
 'Учитель-дефектолог',
 'Учитель-логопед']

In [25]:
isp = df[df['Работники непосредственно осуществляющие и обеспечивающие основной учебно-вспомогательный процесс во взаимодействии с детьми'] == 1]['Должность'].tolist()
isp

['Преподаватель',
 'Воспитатель',
 'Инструктор по физической культуре',
 'Концертмейстер',
 'Мастер производственного обучения',
 'Музыкальный руководитель',
 'Педагог дополнительного образования',
 'Педагог-организатор',
 'Педагог-психолог ',
 'Преподаватель ',
 'Социальный педагог',
 'Тьютор',
 'Учитель',
 'Учитель-дефектолог',
 'Учитель-логопед',
 'Помощник воспитателя',
 'Младший воспитатель']

In [51]:
def szp_ped(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ped:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0 or pd.isna(row['job_' + months[-1]]):
        return np.nan
    return sum_szp / cnt

In [52]:
res['szp_ped'] = res.apply(lambda row: szp_ped(row, ['jan', 'feb']), axis=1)

In [53]:
res

,inn,snils,sum_jan,job_jan,sum_feb,job_feb,szp,szp_ped
0,5003021368,001-191-281 85,106626.88,Учитель,139753.92,Учитель,123190.40,123190.40
1,5003021368,001-244-107 65,109394.87,Учитель,109541.13,Учитель,109468.00,109468.00
2,5003021368,001-263-234 77,74520.00,Воспитатель,81178.00,Воспитатель,77849.00,77849.00
3,5003021368,001-322-594 80,83013.52,Специалист,75815.00,Специалист,79414.26,NaN
4,5003021368,001-519-663 10,142589.38,Учитель,163810.32,Учитель,153199.85,153199.85
...,...,...,...,...,...,...,...,...
130102,9715217689,173-260-871 66,NaN,NaN,67358.00,Воспитатель,67358.00,67358.00
130103,9715217689,184-638-768 23,NaN,NaN,142971.65,Педагог-организатор,142971.65,142971.65
130104,9715217689,184-820-648 92,NaN,NaN,79250.00,Специалист по закупкам,79250.00,NaN
130105,9715217689,187-933-260 19,NaN,NaN,125727.05,Учитель,125727.05,125727.05


In [67]:
val_5 = res[['inn', 'szp_ped']].groupby('inn').mean()
val_5['value'] = val_5['szp_ped'] / depr_month_inc
val_5

,szp_ped,value
inn,,
5003021368,100244.346767,1.191966
5003021495,107910.061588,1.283116
5003021640,117003.647170,1.391244
5003096290,99680.673474,1.185264
5030032182,105850.473651,1.258626
...,...,...
7743939315,104540.877500,1.243054
7743940590,134073.819858,1.594219
9705101759,NaN,NaN


In [66]:
val_6 = res[['inn', 'szp_ped']].groupby('inn').quantile(0.5).rename(columns={'szp_ped': 'q'})
val_6 = pd.merge(val_6, res[['inn', 'szp_ped']], how='right', on='inn')
val_6 = val_6[val_6.szp_ped <= val_6.q][['inn', 'szp_ped']].groupby('inn').mean()
val_6

,szp_ped
inn,
5003021368,74118.139286
5003021495,85147.774688
5003021640,86247.207554
5003096290,71083.491294
5030032182,76387.475952
...,...
7743937357,98423.758214
7743939315,80638.806429
7743940590,109427.110141


In [76]:
cnt1 = res[res.job_feb == 'Учитель'][['inn', 'szp_ped']].groupby('inn').count().rename(columns= {'szp_ped': 'cnt1'})
cnt1

,cnt1
inn,
5003021368,175
5003021495,173
5003021640,177
5003096290,124
5030032182,47
...,...
7743937357,24
7743939315,20
7743940590,93


In [77]:
cnt2 = res[(res.job_feb == 'Учитель') & (res.szp >= 84100)][['inn', 'szp_ped']].groupby('inn').count().rename(columns= {'szp_ped': 'cnt2'})
cnt2

,cnt2
inn,
5003021368,174
5003021495,173
5003021640,177
5003096290,124
5030032182,46
...,...
7743937357,24
7743939315,20
7743940590,93


In [78]:
val_7 = pd.merge(cnt1, cnt2, how='outer', on='inn')
val_7['val'] = val_7.cnt2 / val_7.cnt1
val_7[val_7.val < 1]

,cnt1,cnt2,val
inn,,,
5003021368,175,174,0.994286
5030032182,47,46,0.978723
5051005623,41,40,0.975610
5074045703,135,132,0.977778
7701905508,59,58,0.983051
...,...,...,...
7743868110,53,49,0.924528
7743917311,70,69,0.985714
7743928539,17,16,0.941176


In [79]:
out = res[['inn', 'szp_ped']].groupby('inn').count().rename(columns={'szp_ped': 'Количество педов'})
out

,Количество педов
inn,
5003021368,433
5003021495,447
5003021640,371
5003096290,285
5030032182,126
...,...
7743939315,42
7743940590,141
9705101759,0


In [81]:
df

,inn,snils,sum,job,day,type,stv,status
0,7734408766,001-879-913 57,119742.67,Преподаватель,0,Основное место работы,1.0,Работа
1,7734408766,001-896-420 39,142964.10,Преподаватель,0,Основное место работы,1.0,Работа
2,7734408766,002-050-383 67,68007.79,Преподаватель,1,Основное место работы,1.0,Работа
3,7734408766,002-065-271 85,91097.30,Преподаватель,0,Основное место работы,1.0,Работа
4,7734408766,002-845-525 25,114596.10,Преподаватель,0,Основное место работы,1.0,Работа
...,...,...,...,...,...,...,...,...
199468,7722069597,186-373-917 14,67494.74,Специалист по интернет-маркетингу,0,Основное место работы,1.0,Работа
199469,7722069597,187-703-552 01,62968.42,Воспитатель,0,Основное место работы,1.0,Болезнь
199470,7722069597,198-391-256 24,91973.89,Воспитатель,1,Основное место работы,1.0,Работа
199471,7722069597,202-693-820 52,73130.37,Учитель,0,Основное место работы,1.0,Болезнь


In [82]:
out = df[df.job.isin(ped)].drop_duplicates(['inn', 'snils'])[['inn', 'snils']].groupby('inn').count().rename(columns={'snils':'Количество педов'})

In [84]:
out
wb = Workbook()
ws = wb.active
for  i in df_to_row(out, header=True):
    ws.append(i)
wb.save('peds.xlsx')

In [ ]:
school_sum = sample[['inn', 'sum']].groupby('inn').sum()

In [ ]:
school_sum

In [ ]:
s = sums.groupby(['snils']).count().reset_index()
wb = Workbook()
ws = wb.active
for  i in df_to_row(s, header=True):
    ws.append(i)
wb.save('test.xlsx')